<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ резюме из HeadHunter
   

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

KeyboardInterrupt: 

# Исследование структуры данных

1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла. 

In [ ]:
origin_headhunter_data = pd.read_csv("dst-3.0_16_1_hh_database.csv",  sep=';')


2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [ ]:
headhunter_data = origin_headhunter_data.copy()
headhunter_data.head(3)

3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.

4. Обратите внимание на информацию о числе непустых значений.

In [ ]:
headhunter_data.info()

5. Выведите основную статистическую информацию о столбцах.


In [ ]:
headhunter_data.describe()

# Преобразование данных

1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

*Совет: проверяйте полученные категории, например, с помощью метода unique()*


In [ ]:
# Добавляем столбец 'Образование'. Используем начальные буквы для определения образования. Задаем тип данных category.

headhunter_data['Образование'] = headhunter_data['Образование и ВУЗ'].apply(lambda x:
                                                                            'высшее' if str(x)[0]=='В'
                                                                            else ('неоконченное высшее' if str(x)[0]=='Н'
                                                                                  else ('среднее специальное' if str(x)[8]=='с'
                                                                                        else 'среднее')))
headhunter_data['Образование'] = headhunter_data['Образование'].astype('category')


headhunter_data = headhunter_data.drop('Образование и ВУЗ', axis='columns')
headhunter_data['Образование'].value_counts()

2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979 
* Женщина , 21 год , родилась 13 января 2000
Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина. 
* Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. 
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*


In [ ]:
headhunter_data['Пол'] = headhunter_data['Пол, возраст'].apply(lambda x: str(x)[0]).astype('category')
headhunter_data['Возраст'] = headhunter_data['Пол, возраст'].apply(lambda x: str(x)[11:13]).astype('int')
headhunter_data = headhunter_data.drop('Пол, возраст', axis='columns')

women = headhunter_data['Пол'].value_counts()['Ж'] / headhunter_data.shape[0]
age_mean = headhunter_data['Возраст'].mean()
display(f"Доля женских резюме: {round(women,2)}")
display(f"Средний возраст: {round(age_mean, 1)}")


In [ ]:
display(f"Процу женских резюме: {women:.2f}")
display(f"Средний возраст: {age_mean:.1f}")

3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**. 

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
* В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
    * Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [ ]:
headhunter_data['Опыт работы (месяц)'] = headhunter_data['Опыт работы'].apply(lambda x: np.nan if x is np.nan or x == 'Не указано'
                                                                              else str(x)[12:30])  
def experience_func(expirience_text):
    if expirience_text is np.nan:
        return np.nan
    else:
        experience = 0
        experience_list = expirience_text.rsplit(sep = " ")
        months = ['месяц', 'месяцев', 'месяца']
                
        if experience_list[1] in months:
            experience += int(experience_list[0])
            return experience
        else:
            experience += int(experience_list[0]) * 12
            
            if (len(experience_list)>3) and (experience_list[3] in months):
                experience += int(experience_list[2])
            
        return experience
    
headhunter_data['Опыт работы (месяц)'] = headhunter_data['Опыт работы (месяц)'].apply(experience_func)

headhunter_data = headhunter_data.drop(columns = ['Опыт работы'])
display(f"Медианный опыт работы в месяцах: {headhunter_data['Опыт работы (месяц)'].median()}")

4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:
    
   <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , … 
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.
    
Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*

In [ ]:
# Создаем столбец City_move ['Город', 'Метро'(если есть), 'Переезд', 'Коммандировки'(если есть)]. Проверяем правильность преобразования.

headhunter_data['City_move'] = headhunter_data['Город, переезд, командировки'].apply(lambda x: x.rsplit(sep=' , '))
headhunter_data['City_move_lenth'] = headhunter_data.City_move.apply(lambda x: len(x))
headhunter_data.City_move_lenth.value_counts()

In [ ]:
# Одна из записей не стандатна и содержит 5 элементов. Находим и приводим к стандарту.
display(headhunter_data.City_move[headhunter_data['City_move_lenth'] == 5].values)
headhunter_data.loc[17787, 'City_move'].remove('Колпино)')


In [ ]:
# Создаем признак "Город"
million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 
                  'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград']

headhunter_data['Город'] = headhunter_data['City_move'].apply(lambda x: x.pop(0))
headhunter_data['Город'] = headhunter_data['Город'].apply(lambda city: 
                                                          city if city in ['Москва', 'Санкт-Петербург']
                                                          else ('город-миллионник' if city in million_cities 
                                                                else 'другие'))

saint_petersburg = headhunter_data[headhunter_data['Город'] == 'Санкт-Петербург'].shape[0] / headhunter_data.shape[0]
display(f'Доля соискателей из Санкт-Петербурга: {round(saint_petersburg, 2)}')

In [ ]:
# Создаем признак 'Готовность к перезду'. Сначала где нужно убираем индекс из City_move. 
# Остается ['Переезд', 'Коммандировки'(если есть)]

headhunter_data['City_move'] = headhunter_data['City_move'].apply(lambda x: x if len(x) < 3 else x[1:])

headhunter_data['Готовность к переезду'] = headhunter_data['City_move'].apply(lambda x: False if 'не' in x[0] else True)

headhunter_data['Готовность к командировкам'] = headhunter_data['City_move'].apply(lambda x: False if len(x)<2
                                                                                   else (False if 'не' in x[1] else True))

headhunter_data = headhunter_data.drop(columns=['City_move', 'City_move_lenth','Город, переезд, командировки'])

movers = headhunter_data[headhunter_data['Готовность к переезду'] 
                         & headhunter_data['Готовность к командировкам']].shape[0] / headhunter_data.shape[0]

display(f"Доля соискателей готовых и к переездам, и к командировкам: {round(movers, 2)}")

5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [ ]:
# Создаем новые столбцы по схеме One Hot Encoding

headhunter_data['полная занятость'] = headhunter_data['Занятость'].apply(lambda x: True if 'полная занятость' in x else False)
headhunter_data['частичная занятость'] = headhunter_data['Занятость'].apply(lambda x: True if 'частичная занятость' in x else False)
headhunter_data['проектная работа'] = headhunter_data['Занятость'].apply(lambda x: True if 'проектная работа' in x else False)
headhunter_data['стажировка'] = headhunter_data['Занятость'].apply(lambda x: True if 'стажировка' in x else False)
headhunter_data['волонтерство'] = headhunter_data['Занятость'].apply(lambda x: True if 'волонтерство' in x else False)

headhunter_data['полный день'] = headhunter_data['График'].apply(lambda x: True if 'полный день' in x else False)
headhunter_data['сменный график'] = headhunter_data['График'].apply(lambda x: True if 'сменный график' in x else False)
headhunter_data['гибкий график'] = headhunter_data['График'].apply(lambda x: True if 'гибкий график' in x else False)
headhunter_data['удаленная работа'] = headhunter_data['График'].apply(lambda x: True if 'удаленная работа' in x else False)
headhunter_data['вахтовый метод'] = headhunter_data['График'].apply(lambda x: True if 'вахтовый метод' in x else False)

headhunter_data = headhunter_data.drop(columns = ['Занятость', 'График'])

display(f"Проeктная работа и волонтерство:{headhunter_data[headhunter_data['проектная работа'] & headhunter_data['волонтерство']].shape[0]}")
display(f"Вахтовый метод и гибкий график:{headhunter_data[headhunter_data['вахтовый метод'] & headhunter_data['гибкий график']].shape[0]}")


6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**. 
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата, 
* "proportion" - пропорция, 
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>


Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования: 
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
# Добавляем столбец date - дата обновления резюмме. Преобразуем в формат datetime
headhunter_data['date'] = headhunter_data['Обновление резюме'].apply(lambda x: str(x)[:10])
headhunter_data['date'] = pd.to_datetime(headhunter_data['date'], format="%d.%m.%Y")

# Создаем DataFrame с курсами валют из csv. Задаем формат datetime для столбца date. Удаляем лишние столбцы.
exchange_rates= pd.read_csv('ExchangeRates.csv', sep=',')
exchange_rates.date = pd.to_datetime(exchange_rates.date)
exchange_rates = exchange_rates.drop(columns = ['time', 'vol', 'per'])

In [ ]:
# Создаем столбцы: salary - зарплата (тип int), сurrency - название валюты. С помощью dict конвертируем в кодировку ISO.

headhunter_data['salary'] = headhunter_data['ЗП'].apply(lambda x: x.rsplit(sep=' ')[0]).astype('int')
headhunter_data['currency'] = headhunter_data['ЗП'].apply(lambda x: x.rsplit(sep=' ')[1])

headhunter_data.currency.unique()
exchange_rates['currency'].unique()
replace_dict = {'руб.':'RUB', 'бел.руб.':'BYN', 'KZT':'KZT', 'EUR':'EUR', 'USD':'USD', 'грн.':'UAH', 'сум':'UZS', 'KGS':'KGS',
       'AZN':'AZN'}

headhunter_data.currency = headhunter_data.currency.replace(replace_dict)

# Присоединяем курсы валют методом merge.
headhunter_data = headhunter_data.merge(exchange_rates, how='left', on=['currency', 'date'])

# Для зарплаты в рублях определяем значение close и proportion равным 1
headhunter_data['close'] = headhunter_data['close'].where(cond=headhunter_data.currency != 'RUB', other=1)
headhunter_data['proportion'] = headhunter_data['proportion'].where(cond=headhunter_data.currency != 'RUB', other=1)

# Создаем столбец "ЗП (руб)"
headhunter_data['ЗП (руб)'] = headhunter_data.salary * headhunter_data.close / headhunter_data.proportion

salary_median = headhunter_data['ЗП (руб)'].median()
display(f"Медианная зарплата соискателей: {round(salary_median/1000)} тысяч рублей")

In [ ]:
#Удаляем лишние столбцы
headhunter_data = headhunter_data.drop(columns = ['ЗП', 'date', 'salary', 'currency', 'close', 'proportion'])

# Исследование зависимостей в данных

1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))

boxplot_age = sns.boxplot(
    data = headhunter_data,
    x = 'Возраст',
    orient = 'h',
    width = 0.9,
    ax=axes[0]
)
boxplot_age.set_title('Распределение соискателей по возрасту', fontsize=15);
boxplot_age.grid();

hist_age = sns.histplot(
    data = headhunter_data,
    x = 'Возраст',
    bins=20,
)
hist_age.set_title('Гистограмма распределения соискателей по возрасту', fontsize=15);
hist_age.set_ylabel('Количество');
hist_age.set_xticks(range(5, 85, 5));

display(f"Модальное значение возраста соискателей: {headhunter_data['Возраст'].mode()[0]} лет")

* Возраст большей части соискателей от 20 до 40 лет.
* Распределение соискателей по возрасту похоже на нормальное с выбросами в старшем возрасте. Что говорит, о более редком пристутвии кандитов с возрастом более 50, 60 лет. Некоторые большие значения возраста возможно объясняются ошибками в данных.
* На боксплоте заметно аномальное значение возраста - 10 лет. Может быть ошибкой в данных.

2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака опыта работы, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))

boxplot_experience = sns.boxplot(
    data = headhunter_data,
    x = headhunter_data['Опыт работы (месяц)'] / 12,
    orient = 'h',
    width = 0.9,
    ax=axes[0]
)
boxplot_experience.set_title('Распределение соискателей по опыту', fontsize=15);
boxplot_experience.grid();
boxplot_experience.set_xlabel('Опыт работы, лет');

hist_experience = sns.histplot(
    data = headhunter_data,
    x = headhunter_data['Опыт работы (месяц)'] / 12,
    bins=20,
    ax=axes[1]
)
hist_experience.set_title('Гистограмма распределения соискателей по опыту', fontsize=15);
hist_experience.set_ylabel('Количество');
hist_experience.set_xlabel('Опыт работы, лет');
hist_experience.set_xticks(range(0, 60, 10));

display(f"Модальное значение опыта работы: {headhunter_data['Опыт работы (месяц)'].mode()[0]/12} лет")
display(f"Максимальное значение опыта работы: {headhunter_data['Опыт работы (месяц)'].max()} месяцев")

* Модальное значение опыта работы 6,75 лет. Минимальное значение опыта 1 месяц. 
* Максимальное из данных таблицы 99 лет. Эту аномалию можно объяснить ошибкой данных.
* Опыт большинства соискателей в диапазоне от 0 до 20 лет.



3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака возраста? Обратите внимание на гигантские размеры желаемой заработной платы.
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*


In [ ]:
# Для анализа построим два графика без учета ожидаемых зарплат свыше миллиона рублей. И два графика без учета зарплат выше 400 тыс. руб.

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 10))

boxplot_salary = sns.boxplot(
    data = headhunter_data[headhunter_data['ЗП (руб)'] < 1000000],
    x = 'ЗП (руб)',
    orient = 'h',
    width = 0.9,
    ax=axes[0][0]
)
boxplot_salary.set_title('Распределение соискателей по зарплате \n (зп меньше миллиона)', fontsize=15);
boxplot_salary.grid();
boxplot_salary.set_xlabel('Ожидаемая заработная плата в рублях');

hist_salary = sns.histplot(
    data = headhunter_data[headhunter_data['ЗП (руб)'] < 1000000],
    x = 'ЗП (руб)',
    bins=20,
    ax=axes[0][1]
)
hist_salary.set_title('Гистограмма распределения соискателей по зарплате \n (зп меньше миллиона)', fontsize=15);
hist_salary.set_ylabel('Количество');
hist_salary.set_xlabel('Ожидаемая заработная плата в рублях');


boxplot_salary_l = sns.boxplot(
    data = headhunter_data[headhunter_data['ЗП (руб)'] < 400000],
    x = 'ЗП (руб)',
    orient = 'h',
    width = 0.9,
    ax=axes[1][0]
)
boxplot_salary_l.set_title('(зп меньше 400 тыс. рублей)', fontsize=12);
boxplot_salary_l.grid();
boxplot_salary_l.set_xlabel('Ожидаемая заработная плата в рублях');

hist_salary_l = sns.histplot(
    data = headhunter_data[headhunter_data['ЗП (руб)'] < 400000],
    x = 'ЗП (руб)',
    bins=20,
    ax=axes[1][1]
)
hist_salary_l.set_title('(зп меньше 400 тыс. рублей)', fontsize=12);
hist_salary_l.set_ylabel('Количество');
hist_salary_l.set_xlabel('Ожидаемая заработная плата в рублях');

display(f"Модальное значение опыта работы: {headhunter_data['ЗП (руб)'].mode()[0]} рублей")


In [ ]:
display(headhunter_data[headhunter_data['ЗП (руб)'] > 1000000])

* Модальное значение зарплаты 50 тысяч рублей.
* Минимальная запрашиваемая зарплата в данных 1 рубль, максимальная 24 миллиона 304 тысячи 876 рублей.
* В 84 случаях указана зарплата ниже 1000 руб. Что можно трактовать как ошибку данных, либо как не желание соискателя указывать зарплату.
* Скорее всего объясняется ошибкой запись с требованием зарплаты в 24 миллиона рублей.
* Ожидаемая зарплата большинства соисктелей находится в диапазоне от 40 до 90 тысяч рублей.

4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
salary_education = headhunter_data[headhunter_data['ЗП (руб)'] < 1000000].groupby('Образование')['ЗП (руб)'].median()

fig = plt.figure(figsize=(9, 5))

barplot_salory_ed = sns.barplot(
    x = salary_education.index,
    y = salary_education.values,
);
barplot_salory_ed.set_title('Ожидаемая зарплата в разрезе образования', fontsize=15);

barplot_salory_ed.set_xlabel('Уровень образования');
barplot_salory_ed.set_ylabel("Медианная запрашиваемая зарплата в рублях")
barplot_salory_ed.grid();

* Ожидаемая заработная плата максимальна для соискателей с высшим образованием и минимальна для соискателей со средним и средним специальным образованием.
* Признак уровня образования играет важную роль при прогнозировании заработной платы.

5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
*Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
salary_cities = headhunter_data[headhunter_data['ЗП (руб)'] < 1000000].groupby('Город')['ЗП (руб)'].median()

fig = plt.figure(figsize=(9, 5))

barplot_salary_cities = sns.barplot(
    x = salary_cities.index,
    y = salary_cities.values,
    palette = 'dark'
);
barplot_salary_cities.set_title('Ожидаемая зарплата в разрезе городов', fontsize=15);

barplot_salary_cities.set_ylabel("Медианная запрашиваемая зарплата в рублях")
barplot_salary_cities.grid();

* Наибольший уровень ожидаемой заплаты в Москве, далее Санк-Петербург, наименьший в других городах
* Город является одним из определяющих факторов при прогнозироании зарплаты.

6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.

In [ ]:
fig = plt.figure(figsize=(9, 7))

barplot_salary_movers = sns.barplot(
    data = headhunter_data,
    x = 'Готовность к переезду',
    y = 'ЗП (руб)',
    palette = 'dark',
    hue = 'Готовность к командировкам'
);
barplot_salary_movers.set_title('Зависимость ожидаемой зарплаты от мобильности', fontsize=15);
barplot_salary_movers.set_ylabel("Медианная запрашиваемая зарплата в рублях")
barplot_salary_movers.set_xticklabels(['Не готов', 'Готов']);
barplot_salary_movers.grid();

* Готовность к перезду, как и готовность к командировкам значительно увеличивает прогнозируему заработную плату.

7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.

In [ ]:
age_education_salary = headhunter_data[headhunter_data['ЗП (руб)'] < 1000000].pivot_table(values='ЗП (руб)', index='Образование',
                                                                                          columns='Возраст', aggfunc='median')

fig = plt.figure(figsize=(10, 5))
heatmap = sns.heatmap(data=age_education_salary, cmap='YlGn', annot=False,
                      cbar_kws={'label': 'ожидаемая зарплата в рублях'});
heatmap.set_title('Тепловая карта зависимости медианной зарплаты \n от возраста и образования', fontsize=16);

Постройте сводную таблицу, иллюстрирующую зависимость медианной желаемой заработной платы от возраста («Возраст») и образования («Образование»).

По полученной сводной таблице постройте тепловую карту.

Проанализируйте тепловую карту, сравнив показатели внутри групп.

* На тепловой карте можно увидеть, что высшее образование может двать больший прирост ожидаемой заплаты по мере взросления. 

8. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
fig = plt.figure(figsize=(9, 5))
experience_age_scatterplot = sns.scatterplot(
    data=headhunter_data, 
    y=headhunter_data['Опыт работы (месяц)'] / 12, 
    x='Возраст',
    hue='Образование',
    palette="deep"
);
experience_age_scatterplot.set_title('Зависимость опыта работы от возраста', fontsize=15);
experience_age_scatterplot.set_xlabel('Возраст, лет');
experience_age_scatterplot.set_ylabel('Опыт работы, лет');
experience_age_scatterplot.plot([0, 100], [0, 100], color='grey');

* На графике прослеживается, как и ожидалось, зависимость опыта работы от возраста.
* Заметно несколько аномалий, лежащих выше серой линии, когда соискатель указал опыт работы больше своего возраста.
* Соискатели с вышим образованием в чаще имеют больший опыт работы, чем соискатели со средним специальным образованием.

**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
# Исследуем готовность соискателей к вахтовому методу работы

fig = plt.figure(figsize=(9, 5))
barplot_move_cities = sns.barplot(
    data=headhunter_data,
    x = 'Город',
    y = 'вахтовый метод',
    palette = 'dark'
);
barplot_move_cities.set_title('Готовность к вахтовому методу работы в разрезе городов', fontsize=15);
barplot_move_cities.set_ylabel("Доля готовых к вахтовому методу работы")
barplot_move_cities.grid();

In [ ]:
fig = plt.figure(figsize=(9, 5))

barplot_move_s= sns.barplot(
    data=headhunter_data,
    x = 'Пол',
    y = 'вахтовый метод',
    palette = 'bright'
);
barplot_move_s.set_title('Готовность к вахтовому методу работы среди мужчин и женщин', fontsize=15);
barplot_move_s.set_ylabel("Доля готовых к вахтовому методу работы");
barplot_move_s.set_xlabel(None);
barplot_move_s.set_xticklabels(['Женщины', 'Мужчины']);
barplot_move_s.grid();

* Готовность к вахтовому методу работы выше среди мужчин и соискателей из небольших городов.

# Очистка данных

1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их. 

In [ ]:
# Выяснилось, что когда преобразовывали зарплату в рубли, изменили столбец 'Обновление резюме' навсегда обрезов из него время.
# Изменяем 'Обновление резюме' оставив в нем только дату. Это столбец влияет на количество полных дубликатов.
# Выявляем полные дубликаты и удаляем их.

headhunter_data['Обновление резюме'] = headhunter_data['Обновление резюме'].apply(lambda x: str(x)[:10])
headhunter_data['Обновление резюме'] = pd.to_datetime(headhunter_data['Обновление резюме'], format="%d.%m.%Y")

display(f"Количество дубликатов {headhunter_data[headhunter_data.duplicated()].shape[0]}")
headhunter_data = headhunter_data.drop(headhunter_data[headhunter_data.duplicated()].index)

In [ ]:
headhunter_data = headhunter_data.drop(headhunter_data[headhunter_data.duplicated()].index)

2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах. 

In [ ]:
display(headhunter_data.isna().sum())

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
headhunter_data = headhunter_data.drop(headhunter_data[headhunter_data['Последнее/нынешнее место работы'].isna() 
                                                       | headhunter_data['Последняя/нынешняя должность'].isna()].index)

In [ ]:
headhunter_data['Опыт работы (месяц)'] = headhunter_data['Опыт работы (месяц)'].fillna(headhunter_data['Опыт работы (месяц)'].median())

experience_mean = headhunter_data['Опыт работы (месяц)'].mean()
display(f"Cреднее значение в столбце «Опыт работы (месяц)» после заполнения пропусков: {round(experience_mean)}")

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
headhunter_data = headhunter_data.drop(headhunter_data[(headhunter_data['ЗП (руб)'] > 1000000)
                                                       | (headhunter_data['ЗП (руб)'] < 1000)].index)

5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
headhunter_data = headhunter_data.drop(headhunter_data[(headhunter_data['Опыт работы (месяц)'])/12 > headhunter_data['Возраст']].index)

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

In [ ]:
fig = plt.figure(figsize=(9, 5))
hist_age_log = sns.histplot(
    data = headhunter_data,
    x = 'Возраст',
    log_scale = True,
    bins=20
)
hist_age_log.set_title('Гистограмма логарифмического распределения \n соискателей по возрасту', fontsize=15);
hist_age_log.set_ylabel('Количество');
hist_age_log.set_xlabel('Возраст в логарифмической шкале');

age_mean = np.log(headhunter_data['Возраст']).mean() # среднее логарифмированных данных
age_sigma = np.log(headhunter_data['Возраст']).std() # сигма логарифмированных данных

hist_age_log.axvline(np.exp(age_mean), color='r', lw=2);
hist_age_log.axvline(np.exp(age_mean  + age_sigma * 3), linestyle='--', color='r', lw=2);
hist_age_log.axvline(np.exp(age_mean  - age_sigma * 3), linestyle='--', color='r', lw=2);

* Логарифмическое распределение возраста соискателей асимметрично. В сторону увеличения возраста плавное снижение, с наличеме больших возрастов выходящих за пределы трех сигма. Возраст ниже среднего имеет дополнительные пики и затем резко снижается, в массе не выходя за границы трех сигм.

In [ ]:
def outliers_z_score_mod(data, feature, log_scale=False,  # функция z-отклонений с опциями логарифмической шкалы, и количества сигм
                        left=3, right=3):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned  = outliers_z_score_mod(headhunter_data, 'Возраст', log_scale=True, left=3, right=4)

display(outliers)
headhunter_data = headhunter_data.drop(outliers.index);

*  В таблицу выбросов попали записи с возрастом 15 лет и младше.